# Topic Extraction in Newsfeeds
* Autor: Prof. Dr. Johannes Maucher
* Datum: 17.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)

# Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __RSS Feeds:__ Struktur von RSS Feeds analysieren und parsen mit dem _Universal Feed Parser_. 
* __Dokument Analyse:__ Die Häufigkeit aller Worte in einem Dokument (Inhalt des RSS Feeds) zählen und in einem Array verwalten. 
* __Merkmalsextraktion:__ Bestimmung von Merkmalen (hier auch: __Topics__) (Allgemein spricht man von Merkmalen. Im Fall, dass die NNMF auf Dokumente angewandt wird, werden die Merkmale auch mit __Topics__ oder __Themen__ bezeichnet) mit der \emph{Non Negative Matrix Factorization}.
* __Zuordnung__: Wie setzen sich die Topics aus den Wörtern zusammen? Wie stark sind die gefundenen Topics in den Artikeln vertreten?
* __Dokument Clustering:__ Mit der NNMF kann auch ein Clustering realisiert werden. Jeder Topic repräsentiert ein Cluster. Jedes Dokument wird dem Cluster zugeordnet, dessen Topic am stärksten in ihm vertreten ist. 

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Stellen Sie sich vor Sie möchten in eine eigene Webseite die RSS Feeds einer Menge von Nachrichtenservern einbinden. Da die unterschiedlichen Server wahrscheinlich Artikel zu den gleichen Themen anbieten, werden die Inhalte einiger Artikel ähnlich sein. Mit der __Nicht Negativen Matrixfaktorisierung (NNMF)__ kann für eine große Menge von Dokumenten eine Menge von Themen (Topics) ermittelt werden, auf die sich die Dokumente beziehen. Damit ist es u.a. möglich
* die Dokumente thematisch zu ordnen
* zu jedem Thema nur ein Dokument anzuzeigen

### Ähnlichkeiten bestimmen und relevante Merkmale extrahieren

Eine Sammlung von Dokumenten - in diesem Versuch die Menge aller Nachrichten der angegebenen Feeds - kann in einer Artikel/Wort-Matrix repräsentiert werden. Jede Zeile dieser Matrix gehört zu einem Dokument. Für jedes Wort, das mindestens in einem der Dokumente vorkommt, ist eine Spalte vorgesehen. Das Matrixelement in Zeile $i$, Spalte $j$ beschreibt wie häufig das Wort in Spalte $j$ im zur Zeile $i$ gehörenden Dokument vorkommt.

Unter der Annahme, dass Artikel umso ähnlicher sind, je mehr Worte in diesen gemeinsam vorkommen, kann auf der Grundlage dieser Matrix die Ähnlichkeit zwischen den Artikeln berechnet werden. Hierzu könnte die Matrix z.B. einfach einem _Hierarchischen Clustering_ übergeben werden. Das hierarchische Clustering weist jedoch im Fall einer großen Menge von zu vergleichenden Objekten zwei wesentliche Nachteile auf: Erstens ist die wiederholte Berechnung der Distanzen zwischen allen Artikeln/Clustern extrem rechenaufwendig, zweitens ist die Darstellung einer großen Anzahl von Objekten im Dendrogramm nicht mehr übersichtlich. 

Für das Auffinden von Assoziationen zwischen Dokumenten hat sich in den letzten Jahren die Methode der __Nicht-Negativen Matrix Faktorisierung (NNMF)__ etabliert. Mit dieser Methode kann eine Menge von wesentlichen Merkmalen berechnet werden, anhand derer sich die Dokumente clustern lassen, d.h. Dokumente des gleichen Clusters repräsentieren das gleiche Merkmal (Thema). Ein solches Merkmal wird durch eine Menge von Worten beschrieben, z.B. $\{$ _Paris, terror, IS_ $\}$  oder $\{$_refugee, syria, border_ $\}$. Neben der Merkmalsextraktion stellt die relativ geringe Komplexität einen weiteren Vorteil der NNMF dar. Durch die Darstellung der Artikel/Wort-Matrix als Produkt von 2 Faktormatrizen müssen deutlich weniger Einträge gespeichert werden.

### Nicht Negative Matrixfaktorisierung: Die Idee

Die Artikel/Wort-Matrix wird im Folgenden mit $A$ bezeichnet. Sie besitzt $r$ Zeilen und $c$ Spalten, wobei $r$ die Anzahl der Artikel und $c$ die Anzahl der relevanten Worte in der Menge aller Artikel ist. Durch Multiplikation der Matrix $A$ mit dem Vektor $v$ (_wordvec_: Vektor der alle relevanten Worte enthält) werden die Worte den Artikeln $a$ (_articletitles_: Vektor der alle Artikeltitel enthält) zugeordnet:

$$
a=A*v.
$$

Die Idee der NNMF besteht darin die Matrix $A$ als Produkt zweier Matrizen $W$ und $H$ darzustellen,

$$
A=W*H
$$

wobei alle Elemente in $W$ und $H$ größer oder gleich Null sein müssen. Die Matrixmultiplikation erfordert, dass die Anzahl der Zeilen $m$ in $H$ gleich der Anzahl der Spalten in $W$ sein muss. 
Durch die Faktorisierung der Matrix $A$ wird die Zuordnung der Wörter des Wortvektors $v$  zu den Artikeln des Vektors $a$ in zwei Stufen zerlegt. 

$$
f = H*v
$$
$$
a = W*f 
$$

In der ersten Stufe werden durch die Multiplikation von $v$ mit der Matrix $H$ die Wörter einem sogenannten Merkmalsvektor $f$ mit $m$ Elementen zugewiesen. In der zweiten Stufe werden durch die Multiplikation des Merkmalsvektor $f$ mit der Matrix $W$ die einzelnen Merkmale den Artikeln in $a$ zugeordnet. Die Matrix $H$ definiert also aus welchen Wörtern die Merkmale gebildet werden. Sie wird deshalb __Merkmalsmatrix__ genannt. Die Matrix $W$ hingegen beschreibt mit welchem Gewicht die einzelnen Merkmale in den verschiedenen Artikeln auftreten. Sie wird deshalb __Gewichtungsmatrix__ genannt.

Daraus folgt: Wenn eine Faktorisierung der Matrix $A$ gefunden wird, dann werden damit auch relevante Merkmale, also die Themen, definiert, hinsichtlich derer die Artikel effizient kategorisiert werden. Durch die Matrixfaktorisierung wird eine __Merkmalsextraktion__ realisiert. 

### Berechnung der Matrixfaktoren

Für die Berechnung der Faktoren wurde in [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) eine iterative Methode vorgestellt, die derzeit wohl am häufigsten angewandt wird und auch in dieser Übung implementiert werden soll. Der Algorithmus besteht aus folgenden Schritten:
* Gebe die zu faktorisierende Matrix $A$ ein. $r$ sei die Anzahl der Zeilen und $c$ die Anzahl der Spalten von $A$.
* Wähle die Anzahl $m$ der Merkmale, mit $m<c$. _Tipp:_ Für $m$ sollte zunächst ein Wert im Bereich $15$ bis $30$ gewählt werden.
* Lege eine $m \times c$ Matrix $H$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Lege eine $r \times m$ Matrix $W$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Wiederhole bis maximale Anzahl der Iteration erreicht oder Kosten $k$ unter vordefinierter Schwelle:

	* Berechne aktuelles Produkt $B=W*H$ und bereche die Kostenfunktion 
		$$
			k=\left\| A - B \right\|^2 = \sum\limits_{i,j} \left(A_{i,j} - B_{i,j}\right)^2
		$$ 
	* Anpassung der Matrix $H$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		H_{i,j} := H_{i,j} \frac{(W^T*A)_{i,j}}{(W^T*W*H)_{i,j}}
		$$
        
	* __Nach__ der Anpassung der Matrix $H$: Anpassung der Matrix $W$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		W_{l,i} := W_{l,i} \frac{(A*H^T)_{l,i}}{(W*H*H^T)_{l,i}}
		$$

In [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) ist bewiesen, dass durch die o.g. Anpassungsroutinen die Kosten $k$ monoton abnehmen und in einem Minimum konvergieren. Der Algorithmus ist jedoch nicht optimal weil das gefundene Minimum ein lokales Minimum sein kann.

 ## Vor dem Versuch zu klärende Fragen
 
 * Was versteht man unter Artikel/Wort-Matrix? Wie wird diese im aktuellen Versuch gebildet?

Die Artikel/Wort-Matrix repräsentiert mehrere Dokumente und deren Wörter. Dabei sind die Zeilen die Dokumente und die Spalten alle Wörter, die mindestens einmal in einem der Dokumente auftreten.<br>
Im aktuellen Versuch wird die Artikel/Wort-Matrix durch Nachrichten von RSS-Feeds gebildet. Dabei stellt jede Nachricht ein Dokument in der Matrix dar. In Zelle $d_{i}w_{j}$ steht dann, wie oft Wort $w_{j}$ in Dokument $d_{i}$ vorkommt.

* Wie multipliziert man die Matrix
    $$
    A= \left( \begin{array}{cccc}
a_{00} & a_{01} & a_{02} & a_{03} \\ 
a_{10} & a_{11} & a_{12} & a_{13} \\ 
a_{20} & a_{21} & a_{22} & a_{23}
\end{array} \right)
    $$
    mit dem Vektor  
    $$
    v=\left( \begin{array}{c}
v_{0} \\ 
v_{1} \\ 
v_{2} \\ 
v_{3}
\end{array} \right)
    $$
    

Die Multiplikation der Matrix A mit dem Vektor v sieht wie folgt aus:
$A*v=\left( \begin{array}{c}
a_{00} * v_{0} + a_{01} * v_{1} + a_{02} * v_{2} + a_{03} * v_{3} \\
a_{10} * v_{0} + a_{11} * v_{1} + a_{12} * v_{2} + a_{13} * v_{3} \\
a_{20} * v_{0} + a_{21} * v_{1} + a_{22} * v_{2} + a_{23} * v_{3}
\end{array} \right)$

* Was versteht man im Kontext der NNMF unter
    * Merkmalsmatrix
    * Gewichtsmatrix

Die **Merkmalsmatrix H** definiert, aus welchen Wörtern die Merkmale gebildet werden. Sie ergibt sich aus $m\times c$, mit Merkmalen $m$ und relevanten Wörtern $c$.
Die **Gewichtsmatrix W** definiert, mit welchem Gewicht die einzelenen Merkmale $m$ in den verschiedenen Artikeln $r$ auftreten. Sie ergibt sich aus $m \times r$.

* Wie werden in Numpy zwei Arrays (Typ numpy.array) 
	* im Sinne der Matrixmultiplikation miteinander multipliziert?
	* elementweise multipliziert?
* Wie wird die Transponierte eines Numpy-Arrays berechnet?

Gegeben seien zwei Numpy Arrays A und B.
- Eine elementweise Multiplikation lässt sich in Numpy mit $C=A*B$ durchführen.
- Eine Matrixmultiplikation lässt sich in Numpy mit np.dot(A, B) durchführen. Dafür muss Matrix B jeoch genauso viele Zeilen haben, wie Matrix A Zeilen hat.

Die Transponierte eines Numpy-Arrays, z.B. Matrix A, lässt sich in Numpy mit np.transpose(A) berechnen.

Alle Imports, die für diesen Versuch benötigt werden, sind hier zusammengefasst.

In [1]:
import feedparser
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from gensim import corpora
import numpy as np
import queue
from IPython.display import display
import pandas as pd

# Versuchsdurchführung
Die in diesem Versuch einzubindenden Feeds sind in der unten stehenden Liste _feedlist_ definiert. Die aus dem vorigen Vesuch bereits bekannte Funktion _stripHTML()_ ist ebenfalls gegeben:

In [2]:
feedlist=['http://feeds.reuters.com/reuters/topNews',
          'http://feeds.reuters.com/reuters/businessNews',
          'http://feeds.reuters.com/reuters/worldNews',
          'http://feeds2.feedburner.com/time/world',
          'http://feeds2.feedburner.com/time/business',
          'http://feeds2.feedburner.com/time/politics',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml',
          'http://feeds.nytimes.com/nyt/rss/Business',
          'http://www.nytimes.com/services/xml/rss/nyt/World.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/Economy.xml'
          ]

In [3]:
def stripHTML(h):
    p=''
    s=0
    for c in h:
        if c=='<': s=1
        elif c=='>':
            s=0
            p+=' '
        elif s==0: p+=c
    return p

## Anlegen der Artikel/Wort-Matrix

### Die Funktion _getarticlewords()_
Schreiben Sie eine Funktion _getarticlewords()_, die folgende Elemente zurückgibt:

* _allwords:_ ist ein Dictionary dessen Keys die Worte aller gesammelten Artikel sind. Der zu jedem Key gehörende Wert ist die Anzahl, wie oft das Wort insgesamt vorkommt.
* _articlewords:_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Listenelement ist ein Dictionary, welches die Worte des jeweiligen Artikels als Key enthält und als Wert die Worthäufigkeit.
* _articletitles_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Element ist der Artikeltitel als String.

Für das Parsing der Feeds soll wieder das Modul _feedparser_ eingesetzt werden. Die zu einer Nachricht gehörenden Wörter sollen die Wörter des Elements _title_ und die Wörter des Elements _description_ sein (siehe voriger Versuch). Allerdings sollen hier nicht alle Wörter eingebunden werden, sondern wie im vorigen Versuch eine Methode _getwords()_ implementiert werden, welche nur die _relevanten_ Wörter zurückgibt. Die Frage welche Wörter relevant sind ist nicht eindeutig beantwortbar. Sie können sich hierzu eigene Antworten einfallen lassen. Auf jeden Fall sollten aber die Stopwörter ignoriert werden. Hierzu kann z.B. die Stopwortliste von NLTK angewandt werden.

Nachdem alle relevanten Wörter aller Nachrichten gesammelt sind, sollte eine weitere Bereinigung stattfinden, die 

* alle Wörter, die weniger als 4 mal vorkommen
* alle Wörter, die in mehr als 30% aller Dokumente vorkommen

entfernt. 

Durch dieses Herausfiltern nicht relevanter Wörter kann es vorkommen, dass einzelne Artikel keine relevanten Wörter mehr enthalten. Diese Artikel sollen dann ganz ignoriert werden. D.h. unter anderem, dass diese Artikel auch nicht in _articlewords_ und _articletitles_ erscheinen.

### Methode getWordList
In der Methode ***getWordList*** wird das übergebene Dokument in seine Wörter zerlegt, Satzzeichen entfernt, ggf. sehr kurze Wörter und sehr lange Wörter entfernt und Stopwörter gefiltert. Die daraus resultierende Wortliste wird zurückgegeben.

In [4]:
def getWordList(doc):
    words = str.split(doc.lower())
    wordsStripped = [word.strip('().,:;!?-"') for word in words]
    words = [word for word in wordsStripped if 2 < len(word) < 20 and
             word not in stopwords.words('english')]
    return words

### Funktion getarticlewords
Initial werden ein Dictionary *allwords*, eine Liste *articlewords* und eine Liste *articletitles* angelegt. Dann wird über alle Feeds der übergebenen *feedList* iteriert, der Feed geparst, und anschließend über jede Nachricht des Feeds iteriert. Für jede Nachricht werden folgende Schritte durchgeführt:
> - Hinzufügen des Nachrichtentitels zur Liste *articletitles*.
- Erstellen einer Wortliste *word* mithilfe der Methode ***getWordList***.
- Die Häufigkeit jedes Worts der Liste wird gezählt und zum einen im Dictionary *allwords* hinzugezählt und zum anderen im Dictionary *docDict* für jedes Dokument abgespeichert.
- Das Dictionary *docDict* wird anschließend in die Liste *articlewors* geschrieben.

Anschließend wird eine Liste *nonrelevantwords* erstellt, die mit allen Wörtern befüllt wird, die seltener als vier mal vorkommen oder in mehr als 30% aller Dokumente vorkommen. Durch iterieren über diese Liste, wird jedes Wort, das in der Liste enthaten ist, aus dem Dictionary *allwords* und den Dictionarys in *articlewords* entfernt.

Dokumente in *articlewords*, die dadurch leer sind (repräsentiert durch ein leeres Dictionary), werden anschließend aus *articlewords* entfernt. Genauso wie der zugehörige Titel aus *articletitles* entfernt wird.

Gleiches führen wir auch durch, wenn das Dictionary in *articlewords* nur noch zwei oder weniger Wörter enthält. Dann ist das Dokument für uns nicht mehr relevant, da z.B. für die Wörter *'joe' 'one'* kein sinnvolles Thema zugeordnet werden kann.

Zurückgegeben werden *allwords*, *articlewords* und *articletitles*.

In [5]:
def getarticlewords(feedList, output=True):
    allwords = {}
    articlewords = []
    articletitles = []
    words = []

    for feed in feedList:
        if output == True:
            print('*' * 30)
            print(feed)
        f = feedparser.parse(feed)
        for message in f.entries:
            if re.match(r'\S', message.title):
                if output == True:
                    print(10 * '-' + message.title + 10 * '-')
                articletitles.append(message.title)
                try:
                    fulltext = stripHTML(message.title + ' ' + message.description)
                    words = getWordList(fulltext)
                except:
                    try:
                        fulltext = stripHTML(message.title + ' ' + message.summary)
                        words = getWordList(fulltext)
                    except:
                        print('no description or summary for', message.title)
                docDict = {}
                for word in words:
                    if word in allwords.keys():
                        allwords[word] = allwords[word] + 1
                    else:
                        allwords[word] = 1
                    if word in docDict.keys():
                        docDict[word] = docDict[word] + 1
                    else:
                        docDict[word] = 1
                articlewords.append(docDict)

    nonrelevantwords = [word for word in allwords.keys() if allwords[word] < 4]
    for word in allwords.keys():
        if word not in nonrelevantwords:
            counter = 0
            for a in articlewords:
                if word in a.keys():
                    counter += 1
            if counter / len(articletitles) > 0.3:
                nonrelevantwords.append(word)

    for word in nonrelevantwords:
        catchValue = allwords.pop(word)
        for article in articlewords:
            catchValue = article.pop(word, None)

    for article in articlewords:
        if not article:
            articletitles[articlewords.index(article)] = None
            articlewords[articlewords.index(article)] = None
        elif len(article) < 3:
            for word in article.keys():
                 allwords[word] -= 1
            articletitles[articlewords.index(article)] = None
            articlewords[articlewords.index(article)] = None

    articlewords = list(filter(None, articlewords))
    articletitles = list(filter(None, articletitles))
    return allwords, articlewords, articletitles

#### Ausgabe der Dictionarys aus articlewords

In [6]:
allwords, articlewords, articletitles = getarticlewords(feedlist)

print(50*'*')
for docDict in articlewords:
    print(docDict)

******************************
http://feeds.reuters.com/reuters/topNews
----------Special Report: Trump's catch-and-detain policy snares illegal immigrants long in U.S.----------
----------Trump says will sign something 'pre-emptive' on immigration border policy----------
----------Exclusive: Pope criticizes Trump administration policy on migrant family separation----------
----------EU to impose duties on U.S. imports Friday after Trump tariffs----------
----------Walt Disney raises bid for Fox assets to $71.3 billion, tops Comcast----------
----------Judge denies Stormy Daniels request to restart case against Trump, Cohen----------
----------U.S. withdrawal leaves vacuum at U.N. rights forum----------
----------Chinese media denounces Trump trade moves as Beijing touts sincerity----------
----------For OPEC, oil tariff spat is short-term gain, long-term pain----------
----------After surrendering CSeries, Bombardier pushes regional plane revival----------
****************************

----------WHO: Transgender people not mentally ill----------
----------Israel warns Hamas as tensions escalate over Gaza rockets and incendiary kites----------
******************************
http://rss.cnn.com/rss/edition_world.rss
----------Can a Chinese law help stop the slaughter in Africa?----------
----------How you can help save the African elephant----------
----------2017: Hundreds of elephants relocated----------
----------Rhinoceros DNA database helps nail poachers and traffickers----------
----------Europe will hit US products with tariffs starting Friday----------
----------China's economy shows signs of slowing. A trade war won't help----------
----------Chinese investment in the United States has plummeted 92% this year----------
----------Shale exec: US will be the world's biggest oil producer by the fall----------
----------Ford and Volkswagen may develop vehicles together----------
----------When his foster kids arrived with trash bags, he did this----------
----------

----------In China Trade War, Apple Worries It Will Be Collateral Damage----------
----------As Trump Escalates Trade Fight, China Can Take the Hit----------
----------C.E.O.s Think the Kim Meeting Was a Bad Deal for Trump----------
----------Audi Names Interim C.E.O. After Rupert Stadler Is Arrested----------
----------Google, Rebuilding Its Presence in China, Invests in Retailer JD.com----------
******************************
http://www.nytimes.com/services/xml/rss/nyt/World.xml
----------On Mexico’s Migrant Trail: Trump Policy Means Tough Choices----------
----------Hungary Passes ‘Stop Soros’ Law Criminalizing Aid to Migrants----------
----------As Kim Ends Beijing Visit, China and North Korea Craft New Messages----------
----------Violence in Nicaragua Undermines Peace Talks 2 Months Into Uprising----------
----------U.N. Panel Condemns ‘Pervasive War Crimes’ in Syria Siege----------
----------Canada Vote on Marijuana Paves the Way for Legalization----------
----------Merkel and M

### Die Funktion _makematrix()_
Schreiben Sie eine Funktion _makematrix()_, die aus dem Dictionary _allwords_ und der Liste _articlewords_ (vorige Aufgabe) die Artikel-/Wort-Matrix generiert. Die Einträge in der Matrix sollen die Häufigkeiten der Wörter im jeweiligen Dokument sein (term frequency tf). Die Artikel-/Wort-Matrix soll als 2-dimensionales Numpy Array angelegt werden.

#### Codebeschreibung
In der Funktion ***makematrix*** wird zuerst ein ***gensim.corpora.Dictionary*** erstellt, dem alle Wörter aus *allwords* übergeben werden. Anschließend  wird aus den Dictionarys der Liste *articlewords*, für jedes Dokument eine Liste mit allen Wörtern erstellt. Wörter können auch mehrfach vorkommen.
Auf Basis des *gensim Dictionarys* und der Wörterliste kann mithilfe der Funktion *doc2bow* eine tf-Liste für jedes Dokument erstellt werden. Dabei ist jedes Element ein Tupel, das sowohl den Index des Wortes im Dictionary, als auch die Häufigkeit des Wortes enthält. Anschließed wird mithilfe der Liste ein zweidimensinales Numpy-Array erstellt, das die BOW-Matrix repräsentiert. Die Matrix und das *gensim Dictionary* werden zurückgegeben.

In [7]:
def makematrix(allwords, articlewords):
    dictionary = corpora.Dictionary([allwords.keys()])
    corpus = []
    for doc in articlewords:
        wordList = []
        for k, v in doc.items():
            wordList += [k] * v
        corpus += [dictionary.doc2bow(wordList)]
    matrix = np.zeros(shape=(len(articlewords), len(allwords.keys())))
    for i in range(len(corpus)):
        for tuple in corpus[i]:
            matrix[i][tuple[0]] = tuple[1]
    return matrix, dictionary

##### Ausgabe der Matrix

In [8]:
matrix, dictionary = makematrix(allwords, articlewords)
display(pd.DataFrame(data=matrix))

,0,1,2,3,4,5,6,7,8,9,...,357,358,359,360,361,362,363,364,365,366
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Die Nicht Negative Matrix Faktorisierung
Die Implementierung der NNMF ist entsprechend der Beschreibung im Theoriekapitel durchzuführen.

* Implementieren Sie die Funktion _cost(A,B)_. Dieser Funktion werden zwei Numpy-Matrizen $A$ und $B$ übergeben. Zurück geliefert werden die nach oben angegebener Formel berechneten Kosten $k$. Diese Funktion wird von der im folgenden beschriebenen Funktion _nnmf(A,m,it)_ benutzt.
* Implementieren Sie die Funktion __nnmf(A,m,it)__. In dieser Funktion soll der oben beschriebene Algorithmus für die Nicht-negative Matrix Faktorisierung ausgeführt werden. Der Funktion wird die zu faktorisierende Matrix $A$, die Anzahl der Merkmale $m$ und die Anzahl der Iterationen $it$ übergeben. Die Funktion gibt die gefundenen Faktoren $W$ und $H$ zurück. In jeder Iteration sollen mit der Funktion __cost(A,B)__ die Kosten berechnet werden. Sobald die Kostenabnahme pro 10 Iterationen kleiner als $2$ ist oder eine maximale Anzahl von Iterationen ($maxIt=200$) erreicht ist, soll der Algorithmus mit der Rückgabe der Faktoren $W$ und $H$ terminieren.     


Tipp für die Implementierung elementweiser Operationen von Matrizen: Für elementweise Operationen müssen in Python/Numpy nicht alle Elemente über Schleifen explizit berechnet werden. Eine elementweise Anpassung aller Matrixelemente kann kompakt programmiert werden indem die beteiligten Matrizen für diese Operationen als Arrays implementiert werden. Sollen z.B. die beiden gleich großen Numpy Arrays $U$ und $V$ elementweise multipliziert werden, dann wäre der entsprechende Programmcode einfach _U*V_.  

### Methode cost
Der Methode werden zwei Matrizen übergeben, für die die Kosten berechnet werden sollen. Zuerst wird geprüft, ob die Matrizen die selbe Form haben. Anschließend werden die Zeilen- und Spaltenanzahl der Matrizen bestimmt. Durch iterieren über die Matrizen wird die quadrierte eulidische Distanz bestimmt. Diese wird zum Schluss zurückgegeben.

In [9]:
def cost(A, B):
    result = 0
    if A.shape == B.shape:
        rows, cols = A.shape
        i = 0
        while i < rows:
            j = 0
            while j < cols:
                result += (A[i][j] - B[i][j]) ** 2
                j += 1
            i += 1
        return result
    else:
        print("Error: matrices have different shapes")
        return 0

### Funktion nnmf
Der Funktion ***nnmf*** wird die tf-BOW-Matrix ***A***, die Anzahl Merkmale, die gefunden werden sollen, und die maximale Anzahl Iterationen übergeben. Nachdem geprüft wurde, ob die angegebene Anzahl Merkmale geringer ist, als die Anzahl Spalten der BOW-Matrix, werden die Matrizen ***H*** und ***W*** mit Zufallswerten erstellt.
Um die Kostenabnahme pro 10 Iterationen bestimmen zu können, verwenden wir eine Queue *costQueue*. In diese wird immer der Kostenwert der aktuellen Iteration hinzugefügt und nach 10 Iterationen der letzte Kostenwert zur Berechnung entnommen. Solange die Anzahl maximaler Iterationen noch nicht erreicht wurde, werden folgende Schritte in einer Schleife ausgeführt:
> - Berechnung der Matrix ***B*** durch eine Matrixmultiplikation der Matrizen ***H*** und ***W***.
- Berechnung der Kosten von ***A*** und ***B*** mithilfe der Methode ***cost***.
- Wenn schon mehr als 10 Iterationen erfolgt sind, Prüfung, ob Kostenabnahme über 10 Iterationen geringer als 2 ist.
    - Wenn ja: Abbruch der Schleife
    - Wenn nein: Fortsetzung der Schleife
- Anpassung der Matrix ***H***.
- Anpassung der Matrix ***W***.

Wenn die Schleife beendet ist, werden die Matrizen ***H*** und ***W*** zurückgegeben.

In [10]:
def nnmf(A, m, it):
    iterations = it
    rows, cols = A.shape
    if m >= cols:
        print("No valid number of features. Must be less than columns.")
        return
    H = np.random.rand(m, cols)
    W = np.random.rand(rows, m)
    costQueue = queue.Queue()
    while it > 0:
        B = np.dot(W, H)
        costs = cost(A, B)
        costQueue.put(costs)
        if it <= iterations - 10:
            if costQueue.get() - costs < 2:
                return H, W
        temp1 = np.array(np.dot(W.transpose(), A))
        temp2 = np.array(np.dot(np.dot(W.transpose(), W), H))
        H = np.array(H * np.true_divide(temp1, temp2))
        W = np.array(W * np.true_divide(np.array(np.dot(A, H.transpose())), np.array(np.dot(np.dot(W, H), H.transpose()))))
        it -= 1
    return H, W

In [11]:
H, W = nnmf(matrix, 20, 200)

## Anzeige der Merkmale und der Gewichte

Im vorigen Abschnitt wurde die Merkmalsmatrix $H$ und die Gewichtsmatrix $W$ berechnet. Diese Matrizen können natürlich am Bildschirm ausgegeben werden, was jedoch nicht besonders informativ ist. Aus den Matrizen können jedoch die Antworten für die folgenden interessanten Fragen berechnet werden:

* In welchen Artikeln sind welche Merkmale stark vertreten?
* Wie lassen sich die insgesamt $m$ Merkmale beschreiben, so dass aus dieser Merkmalsbeschreibung klar wird, welches Thema den Artikeln, in denen das Merkmal stark vertreten ist, behandelt wird? 
 
Die Antwort auf die erste Frage ergibt sich aus der Gewichtsmatrix $W$. Für die Beantwortung der zweiten Frage wird die Merkmalsmatrix $H$ herangezogen.



### Beschreibung der Merkmale

Die Merkmalsmatrix $H$ beschreibt, wie stark die Worte aus _wordvec_ in jedem Merkmal enthalten sind. Jede Zeile von $H$ gehört zu einem Merkmal, jede Spalte von $H$ gehört zu einem Wort in _wordvec_.

Es bietet sich an jedes Merkmal einfach durch die $N=6$ Wörter aus _wordvec_ zu beschreiben, welche am stärksten in diesem Merkmal enthalten sind. Hierzu muss für jedes Merkmal die entsprechende Zeile in $H$ nach den $N=6$ größten Werten durchsucht bzw. geordnet werden. Die entsprechenden Spalten dieser Matrixelemente verweisen dann auf die $N=6$ wichtigsten Worte des Merkmals.

Tipp für die Implementierung: Legen Sie für jedes Merkmal $i$ eine Liste an. Die Listenlänge ist durch die Anzahl der Worte in _wordvec_ (d.h. die Anzahl der Spalten in $H$) gegeben. Jedes Listenelement $j$ enthält selbst wieder 2 Elemente: An erster Stelle den entsprechenden Wert $H_{i,j}$ der Merkmalsmatrix, an der zweiten Stelle das $j.$-te Wort in _wordvec_. Nachdem die Liste angelegt ist, kann sie mit _listname.sort()_ in aufsteigender Reihenfolge sortiert werden. Die abnehmende Sortierung erhält man mit _listname.sort().reverse()_. Danach geben die $N=6$ ersten Listenelemente die für das Merkmal $i$ wichtigsten Worte an.

   
### Präsenz der Merkmale in den Artikeln

Die Gewichtsmatrix $W$ beschreibt, wie stark die $m$ Merkmale in den Artikeln aus _articletitles_ enthalten sind. Jede Zeile von $W$ gehört zu einem Artikel, jede Spalte von $W$ gehört zu einem Merkmal.
Die Berechnung der $M=2$ gewichtigsten Merkmale für jeden Artikel in _articletitles_ kann analog zu der oben beschriebenen Berechnung der $N=6$ wichtigsten Worte eines Merkmals berechnet werden.


### Implementierung

Implementieren Sie eine Funktion _showfeatures(w,h,titles,wordvec)_, welche wie oben beschrieben für jeden Artikel die $M=2$ wichtigsten Merkmale am Bildschirm ausgibt. Dabei soll jedes Merkmal durch die 6 wichtigsten Wörter dieses Merkmals angegeben werden. Siehe Beispielausgabe unten.  

Übergabeparameter der Funktion sind die Merkmalsmatrix $H$, die Gewichtungsmatrix $W$, die Liste aller Artikeltitel _articletitles_ und die Liste aller Worte _wordvec_.


Beispiel fuer Ausgabe:

[(13.54131155883748, 13, u'Putin vows payback after confirmation of Egypt plane bomb'),

(2.2466669548146254, 9, u'Putin vows payback after confirmation of Egypt plane bomb')]

----- ['plane', 'egypt', 'russia', 'month', 'killing', 'putin']

----- ['airport', 'russian', 'crash', 'egypt', 'security', 'officials']

Die Ausgabe ist wie folgt zu interpretieren:
* Für den Artikel _Putin vows payback after confirmation of Egypt plane bomb_ ist 
    * das wichtigste Merkmal durch die 6 Wörter _plane_, _egypt_, _russia_, _month_, _killing_, _putin_ definiert. Das Gewicht dieses Merkmals im Artikel ist 13.54
    * das zweitwichtigste Merkmal durch die 6 Wörter _airport_, _russian_, _crash_, _egypt_, _security_, _officials_ definiert. Das Gewicht dieses Merkmals im Artikel ist 2.24

### Funktion showfeatures
Der Funktion ***showfeatures*** werden die Matrizen ***H*** und ***W***, die Liste aller Nachrichtentitel ***articletitles*** und ein Dictionary ***wordvec***, das die Wörter auf Indices abbildet, übergeben.
Zu Beginn wird für jede Zeile der Merkmalsmatrix ***H*** eine absteigend sortierte Liste aller Wörter und deren Werte, wie stark das Wort im jeweiligen Merkmal enthalten ist, erstellt. Die resultierenden Listen werden alle in der Liste *featurelist* gespeichert. Anschließend wird jede Liste auf die Länge 6 gekürzt, sodass nur die 6 wichtigsten Wörter für jedes Merkmal enthalten sind.
Für die Gewichtsmatrix ***W*** wird ebenfalls eine verschachtelte Liste erstellt, die für jeden Artikel die zwei gewichtigsten Merkmale enthält.
Für jeden Artikel werden anschließend die Worte der Merkmale ausgegeben. 

In [12]:
def showfeatures(H, W, articletitles, wordvec):
    featurelist = []
    for row in H:
        innerList = []
        for key, value in wordvec.items():
            innerList.append((row[value], key))
        innerList.sort(key=lambda tup: tup[0], reverse=True)
        featurelist.append(innerList)
    featurelist = [element[:6] for element in featurelist]

    articlefeatures = []
    for row in W:
        innerList = []
        index = 0
        for index in range(len(featurelist)):
            innerList.append((row[index], index))
        innerList.sort(key=lambda tup: tup[0], reverse=True)
        articlefeatures.append(innerList)
    articlefeatures = [element[:2] for element in articlefeatures]

    for index in range(len(articletitles)):
        print(articletitles[index], articlefeatures[index])
        print('-----------', [x[1] for x in featurelist[articlefeatures[index][0][1]]])
        print('-----------', [x[1] for x in featurelist[articlefeatures[index][1][1]]])
        print(30*'*')

In [13]:
showfeatures(H, W, articletitles, dictionary.token2id)

Special Report: Trump's catch-and-detain policy snares illegal immigrants long in U.S. [(5.190884440187649, 8), (2.331427222154666, 19)]
----------- ['u.s', 'fraud', 'elizabeth', 'holmes', 'theranos', 'last']
----------- ['police', 'known', 'strangers', 'last', 'near', 'case']
******************************
Trump says will sign something 'pre-emptive' on immigration border policy [(13.406864288054065, 18), (5.3882660940686815, 14)]
----------- ['trump', 'president', 'donald', 'something', 'border', 'family']
----------- ['border', 'children', 'parents', 'separated', 'taken', 'asylum']
******************************
Exclusive: Pope criticizes Trump administration policy on migrant family separation [(20.750563601335948, 1), (0.7221646511035409, 7)]
----------- ['trump', 'pope', 'policy', 'migrant', 'administration', 'francis']
----------- ['immigration', 'work', 'employees', 'microsoft', 'tuesday', 'protest']
******************************
EU to impose duties on U.S. imports Friday afte

----------- ['china', 'economy', 'trade', 'states', 'united', 'like']
******************************
Google’s JD.com Deal Shows Silicon Valley Still Wants In on China [(4.3430881494717575, 0), (2.7999979081101247, 2)]
----------- ['kim', 'china', 'north', 'jong-un', 'beijing', 'president']
----------- ['google', 'chinese', 'jd.com', 'retailer', 'company', 'world']
******************************
It Was Supposed to Be an Unbiased Study of Drinking. They Wanted to Call It ‘Cheers.’ [(0.06881796401344367, 12), (0.027442584894871537, 14)]
----------- ['new', 'york', 'said', 'could', 'decision', 'bank']
----------- ['border', 'children', 'parents', 'separated', 'taken', 'asylum']
******************************
Los Angeles Times, Searching for Stability, Names Norman Pearlstine Top Editor [(0.7017226672790731, 0), (0.34415770386155925, 11)]
----------- ['kim', 'china', 'north', 'jong-un', 'beijing', 'president']
----------- ['world', 'people', 'refugee', 'day', 'cup', 'crisis']
**************

## Aufgaben

1. Analysieren Sie die berechneten Topics indem Sie sich überlegen ob die gefundenen 6 Wörter pro Topic wirklich Themen beschreiben.
2. Verändern Sie die Parameter der NNMF (Anzahl der Topics $m$, Anzahl der Iterationen). Bei welcher Einstellung der Parameter erhalten Sie das für sie sinnvollste Resultat (sinnvolle Topics)?
3. Wie kann die _getwords()_ Methode verbessert werden, so dass noch bedeutsamere Topics gefunden werden? 

#### Aufgabe 1 - Analyse der Topics
Das Ergebnis ist teilweise korrekt, jedoch zum größten Teil nicht zur Überschrift passend. Zum Beipiel bekommt ein Artikel mit der Überschrift *'Wild scenes as Mexico shocks Germany'* als Thema die Wörter ['trump', 'friday', 'people', 'wants', 'jong', 'president'] zugeordnet.
Wir haben das Gefühl, dass Artikeln, die aktuelle Themen behandeln, wie z.B. das Treffen Trumps mit Kim, im allgemeinen die Themen besser zugeordnet werden, als Artikeln, die Rahmenereignisse beschreiben, wie z.B. englische Klatschgeschichten.

### Aufgabe 2
Da wir durch Debugging festgestellt haben, dass die *maxIterations* nie erreicht werden, sondern die Schleife der *nnmf* aufgrund von geringer Kostenabnahme bereits nach 60-80 Iterationen abbricht, haben wir keine Versuche mit veränderter Anzahl *maxIterations* vorgenommen. Wir haben nur betrachtet, wie sich das Ergebnis mit unterschiedlicher Merkmalszahl verändert.
#### Erhöhung auf 30 Merkmale
Obwohl die Anzahl der Merkmale um 10 erhöht wurde, konnten wir nicht unbedingt eine Verbesserung der Topic Extraction feststellen.

In [14]:
allwords, articlewords, articletitles = getarticlewords(feedlist, output=False)
matrix, dictionary = makematrix(allwords, articlewords)
H, W = nnmf(matrix, 30, 200)
showfeatures(H, W, articletitles, dictionary.token2id)

no description or summary for Trump makes political gamble with immigrant lives
no description or summary for Nearly 200 feared dead as tourist ferry sinks in world's largest crater lake
no description or summary for World Cup reporter sexually assaulted on live TV
no description or summary for Russia's goal: The world is coming to Putin
no description or summary for Turkey to get F-35 jets despite US Congress opposition
no description or summary for Europe to hit US motorbikes, jeans with tariffs
no description or summary for US expects North Korea to return troop remains
no description or summary for $30M in cryptocurrencies reported stolen
Special Report: Trump's catch-and-detain policy snares illegal immigrants long in U.S. [(4.6912888876693355, 28), (3.231701060060187, 10)]
----------- ['u.s', 'fraud', 'holmes', 'elizabeth', 'theranos', 'friday']
----------- ['policy', 'migrant', 'immigration', 'border', 'separating', 'criticized']
******************************
Trump says will si

Where a Taboo Is Leading to the Deaths of Young Girls [(1.7344896873177105, 26), (1.5245191361688972, 18)]
----------- ['volkswagen', 'ford', 'two', 'biggest', 'stadler', 'forces']
----------- ['european', 'union', 'europe', 'brussels', 'parliament', 'stand']
******************************
That’s ‘Mr. President’ to You: Macron Scolds French Student [(3.642606852378304, 21), (1.9466840952505688, 18)]
----------- ['may', 'parents', 'student', 'day', 'skin', 'according']
----------- ['european', 'union', 'europe', 'brussels', 'parliament', 'stand']
******************************
Do You Know Where Your Pride T-Shirt Was Made? [(0.43532150751534, 0), (0.263675936448899, 17)]
----------- ['world', 'cup', 'beer', 'year', 'migrants', 'museums']
----------- ['fox', 'century', '21st', 'disney', 'offer', 'company']
******************************
Battle Intensifies for Yemeni Port as Dock Workers Still Unload Aid [(2.3161862403836846, 26), (1.3407245099632992, 4)]
----------- ['volkswagen', 'ford'

#### Reduzierung auf 10 Merkmale
Wie erwartet, hat sich das Ergebnis mit 10 Merkmalen nicht verbessert. Das wäre auch nicht logisch gewewsen, da die Feeds ja Artikel zu mehr als 10 Themen enthalten.

In [15]:
allwords, articlewords, articletitles = getarticlewords(feedlist, output=False)
matrix, dictionary = makematrix(allwords, articlewords)
H, W = nnmf(matrix, 10, 200)
showfeatures(H, W, articletitles, dictionary.token2id)

no description or summary for Trump makes political gamble with immigrant lives
no description or summary for Nearly 200 feared dead as tourist ferry sinks in world's largest crater lake
no description or summary for World Cup reporter sexually assaulted on live TV
no description or summary for Russia's goal: The world is coming to Putin
no description or summary for Turkey to get F-35 jets despite US Congress opposition
no description or summary for Europe to hit US motorbikes, jeans with tariffs
no description or summary for US expects North Korea to return troop remains
no description or summary for $30M in cryptocurrencies reported stolen
Special Report: Trump's catch-and-detain policy snares illegal immigrants long in U.S. [(3.450459663201376, 8), (0.7116272681408509, 5)]
----------- ['u.s', 'reuters', 'wednesday', 'european', 'rights', 'union']
----------- ['world', 'people', 'refugee', 'million', 'cup', 'day']
******************************
Trump says will sign something 'pre-em

----------- ['house', 'white', 'trump', 'president', 'nielsen', 'markets']
******************************
U.N. Rights Chief Tells U.S. to Stop Taking Migrant Children From Parents [(4.4980464388758135, 8), (3.606399821498805, 6)]
----------- ['u.s', 'reuters', 'wednesday', 'european', 'rights', 'union']
----------- ['new', 'york', 'border', 'children', 'taken', 'parents']
******************************
Audi Names Interim C.E.O. After Rupert Stadler Is Arrested [(0.5195014385237273, 8), (0.22031499201090085, 6)]
----------- ['u.s', 'reuters', 'wednesday', 'european', 'rights', 'union']
----------- ['new', 'york', 'border', 'children', 'taken', 'parents']
******************************
Google, Rebuilding Its Presence in China, Invests in Retailer JD.com [(5.642112776708242, 5), (4.554818736231972, 3)]
----------- ['world', 'people', 'refugee', 'million', 'cup', 'day']
----------- ['china', 'kim', 'north', 'beijing', 'president', 'jong-un']
******************************
Erdogan’s Plan to

### Aufgabe 3
#### Anwendung von Stemming
Durch die Anwendung des PorterStemmers hat sich das Ergebnis nach unserer Einschätzung verbessert. Das würde auch Sinn machen, da Wörter wie *"trump's"* oder *"germans"* auf ihren Wortstamm abgebildet werden und somit mehr gewichtig bei der Themenzuordnung sind.

In [16]:
def getWordList(doc):
    words = str.split(doc.lower())
    wordsStripped = [word.strip('().,:;!?-"') for word in words]
    words = [word for word in wordsStripped if 2 < len(word) < 20 and word not in stopwords.words('english')]
    words = [PorterStemmer().stem(word) for word in words]
    return words

allwords, articlewords, articletitles = getarticlewords(feedlist, output=False)
matrix, dictionary = makematrix(allwords, articlewords)
H, W = nnmf(matrix, 20, 200)
showfeatures(H, W, articletitles, dictionary.token2id)

no description or summary for Trump makes political gamble with immigrant lives
no description or summary for Nearly 200 feared dead as tourist ferry sinks in world's largest crater lake
no description or summary for World Cup reporter sexually assaulted on live TV
no description or summary for Russia's goal: The world is coming to Putin
no description or summary for Turkey to get F-35 jets despite US Congress opposition
no description or summary for Europe to hit US motorbikes, jeans with tariffs
no description or summary for US expects North Korea to return troop remains
no description or summary for $30M in cryptocurrencies reported stolen
Special Report: Trump's catch-and-detain policy snares illegal immigrants long in U.S. [(3.0448078462851056, 15), (2.3666908027624287, 16)]
----------- ['polic', 'known', 'stranger', 'last', 'say', 'near']
----------- ['tariff', 'opec', 'oil', 'spat', 'reuter', 'u.']
******************************
Trump says will sign something 'pre-emptive' on im

The Week Ahead: Central Bankers Will Debate Policy, and OPEC Meets on Output [(2.7845353768226406, 16), (2.129446220785275, 18)]
----------- ['tariff', 'opec', 'oil', 'spat', 'reuter', 'u.']
----------- ['legal', 'marijuana', 'vote', 'canada', 'tuesday', 'use']
******************************
Feeling Good About the Economy? You’re Probably a Republican [(3.465859651286814, 12), (0.7983360243797641, 1)]
----------- ['trump', 'hous', 'white', 'presid', 'donald', 'meet']
----------- ['china', 'trade', 'economi', 'unit', 'state', 'countri']
******************************
As Uncertainty Clouds Economy, Europe Pulls Back on Easy Money [(3.2001637135584535, 3), (2.0135383336143704, 7)]
----------- ['protest', 'end', 'european', 'plan', 'union', 'europ']
----------- ['new', 'york', 'bank', 'decis', 'said', 'could']
******************************
As China Curbs Borrowing, Growth Shows Signs of Faltering [(6.587356146620135, 1), (2.598009569424312, 6)]
----------- ['china', 'trade', 'economi', 'u

#### Enfernung nicht relevanter Wörter
Folgende Wörter werden in der folgenden Anwendung entfernt, da wir sie unrelevant für die Topic Extraction halten.
> - Namen der Nachrichtenfeeds *(reuters, cnn, nytimes)*
- Zahlen 0-9 ausgeschrieben
- Zahlen, auch mit Währungsangaben

Ob das Ergebnis dadurch wirklich besser wurde, ist für uns schwer zu beurteilen. Es ist jedoch leicht zu erkennen, dass die sechs Wörter, die die Themen beschreiben, aussagekräftiger sind, da sie die entfernten Wörter nicht mehr enthalten können.

In [17]:
def getWordList(doc):
    unrelevantWords = ['reuters', 'cnn', 'nytimes',
                       'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    words = str.split(doc.lower())
    wordsStripped = [word.strip('().,:;!?-"') for word in words]
    words = [word for word in wordsStripped if 2 < len(word) < 20 and word not in stopwords.words('english') and
             word not in unrelevantWords and
             not re.match(r'\$?\d+\.?\d*\€?', word)]
    words = [PorterStemmer().stem(word) for word in words]
    return words

allwords, articlewords, articletitles = getarticlewords(feedlist, output=False)
matrix, dictionary = makematrix(allwords, articlewords)
H, W = nnmf(matrix, 20, 200)
showfeatures(H, W, articletitles, dictionary.token2id)

no description or summary for Trump makes political gamble with immigrant lives
no description or summary for Nearly 200 feared dead as tourist ferry sinks in world's largest crater lake
no description or summary for World Cup reporter sexually assaulted on live TV
no description or summary for Russia's goal: The world is coming to Putin
no description or summary for Turkey to get F-35 jets despite US Congress opposition
no description or summary for Europe to hit US motorbikes, jeans with tariffs
no description or summary for US expects North Korea to return troop remains
no description or summary for $30M in cryptocurrencies reported stolen
Special Report: Trump's catch-and-detain policy snares illegal immigrants long in U.S. [(2.9274829251918364, 4), (2.598631582872568, 3)]
----------- ['polic', 'known', 'stranger', 'last', 'say', 'near']
----------- ['u.', 'fraud', 'elizabeth', 'holm', 'therano', 'charg']
******************************
Trump says will sign something 'pre-emptive' o

In China Trade War, Apple Worries It Will Be Collateral Damage [(3.0453150573609653, 5), (1.615747998751145, 6)]
----------- ['trade', 'market', 'war', 'forc', 'presid', 'call']
----------- ['china', 'trade', 'economi', 'disput', 'state', 'countri']
******************************
As Trump Escalates Trade Fight, China Can Take the Hit [(18.188690194585018, 6), (1.8317550966294305, 10)]
----------- ['china', 'trade', 'economi', 'disput', 'state', 'countri']
----------- ['trump', 'presid', 'immigr', 'donald', 'u.', 'administr']
******************************
C.E.O.s Think the Kim Meeting Was a Bad Deal for Trump [(6.474249761885443, 10), (6.335959836611213, 2)]
----------- ['trump', 'presid', 'immigr', 'donald', 'u.', 'administr']
----------- ['kim', 'china', 'north', 'presid', 'jong-un', 'beij']
******************************
Audi Names Interim C.E.O. After Rupert Stadler Is Arrested [(2.9171681871702764, 4), (0.3067075757598141, 1)]
----------- ['polic', 'known', 'stranger', 'last', 'sa